## 2.1 Tools Calling

In this notebook, we will explore how the LLM can use external tools to enhance its capabilities. We'll begin by adding and configuring a search tool to allow the LLM to perform real-time searches.

By the end of this notebook, you'll be able to integrate and configure external search tools with a language model, and use them to perform real-time searches and enhance responses.

![LLM Tools](../../content/modules/ROOT/assets/images/04/04-02-tools-diagram.png)

Through tool calling, LLMs can:

- **Access Real-Time Data**: Retrieve current information from search engines, databases, or other real-time sources.
- **Perform Specialized Calculations**: Use tools like code interpreters to execute complex calculations or data transformations.
- **Format Responses for Structured Systems**: Generate responses in specific schemas required by APIs or databases, ensuring smooth communication with these systems.

## 1. Setup

#### Installing Required Packages

In [ ]:
!pip install -q langchain openai
!pip install langchain-openai
!pip install "langchain-core==0.3.27"
!pip install langchain_community
!pip install -q termcolor langchain_community duckduckgo_search wikipedia

In [ ]:
# Imports
import os
import json
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

## 2. DuckDuckGo Search Tool

First, we will configure the tool by initializing the DuckDuckGo search functionality. This will be the primary tool for real-time search queries in this example.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo Search Tool
duckduckgo_search = DuckDuckGoSearchRun()

# Verify tool name
print("Search Tool Name:", duckduckgo_search.name)

# Adding the search tool to the list of available tools
tools = [duckduckgo_search]

Search Tool Name: duckduckgo_search


Render the tool’s description and ensures it is formatted correctly for integration into the LLM's prompt.


In [ ]:
# Render tool description to ensure it's ready for LLM usage
from langchain.tools.render import render_text_description_and_args

tools_name = duckduckgo_search.name

tools_description = (
    render_text_description_and_args(tools).replace("{", "{{").replace("}", "}}")
)
print("Tools Name:\n", tools_name)
print("Tools Description:\n", tools_description)

Tools Name:
 duckduckgo_search
Tools Description:
 duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query., args: {{'query': {{'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}}}


## 3. Model Configuration

#### Define the Inference Model Server specifics

In [5]:
INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
MODEL_NAME = "granite-3-8b-instruct"
API_KEY= os.getenv('API_KEY')

#### Create the LLM instance

In [ ]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

## 4. System Prompt for Tool Integration

Defines the system prompt template, which is used to instruct the LLM on how to interact
with external tools. The prompt is structured to guide the LLM in calling tools when necessary.

In [ ]:
template = """\
<|start_of_role|>system<|end_of_role|>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make sense or is not factually coherent, explain why instead of providing incorrect information. If you don't know the answer to a question, do not share false information.

---

You have access to several tools to assist you in completing tasks. You are responsible for determining when to use them and should break down complex tasks into subtasks if necessary. 

When using tools, follow this format:

{{
  "action": "Specify the tool you want to use.",
  "action_input": {{ 
    "key": "Value inputs to the tool in valid JSON format."
  }}
}}

You must always ensure the inputs are correct for the tool you call. Provide all output in JSON format.

Be sure that is in JSON format the output, nothing else. Not repeate the json again.

The available tools include:

{tools_description}

<|end_of_text|>

<|start_of_role|>user<|end_of_role|>
Human: {input}
<|end_of_text|>

<|start_of_role|>assistant<|end_of_role|>
"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

tools_description = (
    render_text_description_and_args(tools)
    .replace("{", "{{")
    .replace("}", "}}")
)

## 5. LLM Request

Here, we issue a user query asking for the latest version of KServe.

In [8]:
user_input = "What are the last 5 year results of the S&P500?"

#### Create the Chain using the different components

In [9]:
#print(tools_description)

conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False,
                        )

/tmp/ipykernel_920/3274098412.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(llm=llm,


The LLM processes the request and responds
with an action calling the DuckDuckGo search tool.

In [10]:
response = conversation.predict(input=user_input, tools_description=tools_description);

{
  "action": "duckduckgo_search",
  "action_input": {
    "query": {
      "title": "Last 5 years S&P 500 results",
      "description": "S&P 500 yearly performance from the past 5 years"
    }
  }
}

## 6. Processing Tool Actions

After the LLM provides its response, we will process the action request. If the LLM calls the DuckDuckGo search tool, we will execute the function and retrieve the results. The tool's output will then be formatted and displayed.

In [11]:
# Parse the LLM's response to extract the action and input
print(f"### LLM Response: {response}")
response_dict = json.loads(response)

action = response_dict.get("action")
action_input = response_dict.get("action_input")

### LLM Response: {
  "action": "duckduckgo_search",
  "action_input": {
    "query": {
      "title": "Last 5 years S&P 500 results",
      "description": "S&P 500 yearly performance from the past 5 years"
    }
  }
}


It then re-feeds the result back into the LLM for further processing (e.g., summarization or extraction of key information).

Finally, we use the `process_tool_and_extract` function to dynamically perform a task (in this case, summarizing and extracting the key information from the search result).

In [ ]:
def process_tool_and_extract(action, action_input, task):
    for tool in tools:
        if tool.name == action:
            print(f"--> Executing tool: {tool.name}")
            try:
                query = action_input['query']
                print(f"----> Executing DuckDuckGo search for: {query}")

                result = tool.invoke(query)
                print(f"------> Search Result: {result}")
                print("\n")

                # Dynamically create a prompt based on the task (summarize or extract information)
                task_prompt = f"Based on the following search result, please {task},{user_input}:\n\n{result}"

                # Feed the search result back into the LLM for summarization or extraction
                print(f"##### Answer to the User Input Question: '{user_input}' #####")
                summary_response = conversation.predict(input=task_prompt, tools_description=tools_description)
                #print(f"Task Result: {summary_response}")
                

            except Exception as e:
                print(f"Error executing tool {action}: {str(e)}")
            break
    else:
        print(f"Tool {action} not found or unsupported operation.")


# Example usage of the function
task = "Summarize and extract the key information of the text. Give me a sentence with that as an answer, not in json, just plain text."
process_tool_and_extract(action, action_input, task)

--> Executing tool: duckduckgo_search
----> Executing DuckDuckGo search for: S&P 500 yearly performance from the past 5 years
------> Search Result: S&P 500 5 Year Return is at 87.83%, compared to 93.58% last month and 54.65% last year. This is higher than the long term average of 46.17%. The S&P 500 5 Year Return is the investment return received for a 5 year period, excluding dividends, when holding the S&P 500 index. 5 year chart of the S&P 500 stock index* The 5 year chart of S&P 500 summarizes the chages in the price well, however, we recommend to have a look at the chart(s) below, too. Similar charts of the past 10 year can be found here. You can find charts of other periods (1, 3, 6, 12 months etc.) here. 5 years return and graph of S&P 500* When looking at the Periods in the Price Performance table, the 5-Day through 2-Year periods are based on daily data, the 3-Year and 5-Year periods are based on weekly data, and the 10-Year and 20-Year periods are based on monthly data. Barc

## LLMs Tools Calling Architecture

![LLM Tools](./llm_tools_llama.jpeg)

1. **User Submits a Prompt**: The process starts when the user provides input or a prompt, which is sent to the **Executor** for handling.
2. **Executor Forwards Prompt to LLM**: The **Executor** forwards the user’s prompt to the **LLM** (LLM) for initial processing.
3. **LLM Processes the Prompt**: The **LLM** analyzes the prompt and determines whether it can generate a response directly or if external tools are needed to complete the request.
4. **LLM Requests External Tool Invocation**: If external data or additional resources are required (e.g., real-time data from a search engine or a code interpreter), the **Model** signals the **Executor** to call the relevant tool.
5. **Executor Calls the Tools**: The **Executor** initiates the tool request, calling external tools (such as Brave Search, Wolfram Alpha, Code Interpreter, or other custom tools) to fetch necessary information.
6. **Tools Retrieve Data**: The external tools access the required information from outside environments (e.g., fetching real-time search results or performing complex calculations) and return the data to the **Executor**.
7. **Executor Synthesizes Final Response**: The **Executor** combines the original prompt, the **LLM**'s processing, and any external tool responses. The final response is then sent back to the **LLM**.
8. **LLM Generates Final Response**: The **LLM** synthesizes all the gathered data and generates a final response.
9. **Executor Sends Response to User**: The **Executor** delivers the final response back to the user, completing the interaction.

## Conclusion
 
In this notebook, we've successfully extended the capabilities of the LLM by integrating an external tool (DuckDuckGo search). We covered:

- **LLM and Tool Integration:** How to modify the system prompt for tool access.
- **Tool Setup and Configuration:** Setting up the DuckDuckGo search tool for real-time information retrieval.
- **Making Requests and Handling Responses:** Using the tool during LLM interactions and processing the results.

With these tools in place, you're now equipped to extend the LLM's capabilities even further by adding more tools and refining its behavior. Happy coding!